## 파이썬으로 말뭉치 전처리 하기

In [1]:
text = 'You say goodbye and I say hello.'
text

'You say goodbye and I say hello.'

In [2]:
text = text.lower()
text

'you say goodbye and i say hello.'

In [3]:
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [4]:
text

'you say goodbye and i say hello .'

In [5]:
words = text.split(' ')

In [6]:
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

###  딕셔너리를 이용하여 단어 ID와 단어를 짝지어 주는 대응표 작성

In [14]:
word_to_id = {}
# print(type(word_to_id))
# print(len(word_to_id))
id_to_word = {}
# print(type(id_to_word))
# print(len(id_to_word))

# print(words)
for word in words:
#     print(word)
#     print(word not in word_to_id)
#     break
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [17]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [18]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [19]:
word_to_id['hello']

5

In [20]:
id_to_word[5]

'hello'

In [22]:
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
print(corpus)

text = [id_to_word[i] for i in corpus]
print(text)

[0 1 2 3 4 1 5 6]
['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']


###  말뭉치를 이용하기 위한 전처리 함수 구현

In [23]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [24]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [25]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [26]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [27]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

### 동시발생 행렬

In [28]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [29]:
print(corpus)

[0 1 2 3 4 1 5 6]


In [30]:
print(id_to_word)

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [31]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0],
], dtype=np.int32)

In [32]:
print(C[0])

[0 1 0 0 0 0 0 0]


In [33]:
print(C[4])

[0 1 0 1 0 0 0 0]


In [34]:
print(C[word_to_id['goodbye']])

[0 1 0 1 0 0 0 0]


In [ ]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

### 벡터 간 유사도

In [35]:
def cos_similarity(x, y):
    nx = x / np.sqrt(np.sum(x ** 2))
    ny = y / np.sqrt(np.sum(y ** 2))
    return np.dot(nx, ny)

In [36]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [42]:
import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, cos_similarity


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size, 3)
# print(C)
c0 = C[word_to_id['you']]  # "you"의 단어 벡터

c1 = C[word_to_id['i']]    # "i"의 단어 벡터

print(c0)
print(c1)
print(cos_similarity(c0, c1))

[0 1 1 1 0 0 0]
[0 2 1 1 0 1 1]
0.8164965733269295


### 유사 단어의 랭킹 표시

In [43]:
import numpy as np
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [44]:
x = np.array([100, -20, 2])

In [45]:
x.argsort()

array([1, 2, 0], dtype=int64)

In [46]:
(-x).argsort()

array([0, 2, 1], dtype=int64)

In [52]:
import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, most_similar

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size, 3)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 i: 0.8164965733269295
 goodbye: 0.6546536644539741
 and: 0.6123724299951971
 say: 0.5962847890176913
 hello: 0.5773502629695412
